## BPTT 
图节选自colah's blog

![](img/bptt.jpg)

$$
s_t=f(U\cdot x +W\cdot s_{t-1})\\
y_t=f(V\cdot s_t)
$$

$$
\frac{\partial \varepsilon}{\partial W}=\sum_t \frac{\partial \varepsilon_t}{\partial W}
$$

对于W来说略有不同，因为$s_t=f(U\cdot x +W\cdot s_{t-1})$,所以对W求偏导，而$s_{t-1}$也是W的函数：
$$
\frac{\partial \varepsilon_t}{\partial W}=\sum_\tau \frac{\partial \varepsilon_t}{\partial y_t}\frac{\partial y_t}{\partial s_t}\frac{\partial s_t}{\partial s_\tau}\frac{\partial s_\tau}{\partial W}
$$

In [4]:
import numpy as np
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation: dL/dz
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            # Add to gradients at each previous step
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step dL/dz at t-1
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]